<a href="https://colab.research.google.com/github/aarav2703/Video-Denoising-Optimization/blob/main/performance_metrics%2BGPU_acceleration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from google.colab import drive
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error

# Mount Google Drive
drive.mount('/content/drive')

# Function to resize frames
def resize_frame(frame, width=None, height=None):
    if width is None and height is None:
        return frame
    if width is None:
        r = height / frame.shape[0]
        dim = (int(frame.shape[1] * r), height)
    else:
        r = width / frame.shape[1]
        dim = (width, int(frame.shape[0] * r))
    resized_frame = cv2.resize(frame, dim, interpolation=cv2.INTER_LINEAR)
    return resized_frame

# Function to compute PSNR
def compute_psnr(video1, video2):
    original = cv2.VideoCapture(video1)
    denoised = cv2.VideoCapture(video2)
    psnr_values = []
    while True:
        ret1, frame1 = original.read()
        ret2, frame2 = denoised.read()
        if not ret1 or not ret2:
            break
        # Resize frames to have the same dimensions
        frame1_resized = resize_frame(frame1, width=frame2.shape[1], height=frame2.shape[0])
        frame1_tensor = tf.convert_to_tensor(frame1_resized, dtype=tf.float32)
        frame2_tensor = tf.convert_to_tensor(frame2, dtype=tf.float32)
        psnr = tf.image.psnr(frame1_tensor, frame2_tensor, max_val=255)
        psnr_values.append(psnr)
    return np.mean(psnr_values)

# Function to compute SSIM
def compute_ssim(video1, video2):
    original = cv2.VideoCapture(video1)
    denoised = cv2.VideoCapture(video2)
    ssim_values = []
    while True:
        ret1, frame1 = original.read()
        ret2, frame2 = denoised.read()
        if not ret1 or not ret2:
            break
        # Resize frames to have the same dimensions
        frame1_resized = resize_frame(frame1, width=frame2.shape[1], height=frame2.shape[0])
        frame1_tensor = tf.convert_to_tensor(frame1_resized, dtype=tf.float32)
        frame2_tensor = tf.convert_to_tensor(frame2, dtype=tf.float32)
        ssim_value = tf.image.ssim(frame1_tensor, frame2_tensor, max_val=255)
        ssim_values.append(ssim_value)
    return np.mean(ssim_values)

# Function to compute MSE
def compute_mse(video1, video2):
    original = cv2.VideoCapture(video1)
    denoised = cv2.VideoCapture(video2)
    mse_values = []
    while True:
        ret1, frame1 = original.read()
        ret2, frame2 = denoised.read()
        if not ret1 or not ret2:
            break
        # Resize frames to have the same dimensions
        frame1_resized = resize_frame(frame1, width=frame2.shape[1], height=frame2.shape[0])
        frame1_tensor = tf.convert_to_tensor(frame1_resized, dtype=tf.float32)
        frame2_tensor = tf.convert_to_tensor(frame2, dtype=tf.float32)
        mse = tf.keras.losses.mean_squared_error(frame1_tensor, frame2_tensor)
        mse_values.append(mse)
    return np.mean(mse_values)

# Define file paths
original_video_path = '/content/drive/MyDrive/Colab Notebooks/video denoising media/noisy original video/skiing.mp4'
spatial_denoised_video_path = '/content/drive/MyDrive/Colab Notebooks/video denoising media/denoised videos/skiing_spatial.mp4'
temporal_denoised_video_path = '/content/drive/MyDrive/Colab Notebooks/video denoising media/denoised videos/skiing_temporal.mp4'
dct_denoised_video_path = '/content/drive/MyDrive/Colab Notebooks/video denoising media/denoised videos/skiing_denoised_video_dct_color_preservation.mp4'

# Compute PSNR, SSIM, and MSE for each pair of videos
psnr_original_spatial = compute_psnr(original_video_path, spatial_denoised_video_path)
psnr_original_temporal = compute_psnr(original_video_path, temporal_denoised_video_path)
psnr_original_dct = compute_psnr(original_video_path, dct_denoised_video_path)

ssim_original_spatial = compute_ssim(original_video_path, spatial_denoised_video_path)
ssim_original_temporal = compute_ssim(original_video_path, temporal_denoised_video_path)
ssim_original_dct = compute_ssim(original_video_path, dct_denoised_video_path)

mse_original_spatial = compute_mse(original_video_path, spatial_denoised_video_path)
mse_original_temporal = compute_mse(original_video_path, temporal_denoised_video_path)
mse_original_dct = compute_mse(original_video_path, dct_denoised_video_path)

# Print results
print("PSNR - Original vs Spatial Denoised:", psnr_original_spatial)
print("PSNR - Original vs Temporal Denoised:", psnr_original_temporal)
print("PSNR - Original vs DCT Denoised:", psnr_original_dct)

print("SSIM - Original vs Spatial Denoised:", ssim_original_spatial)
print("SSIM - Original vs Temporal Denoised:", ssim_original_temporal)
print("SSIM - Original vs DCT Denoised:", ssim_original_dct)

print("MSE - Original vs Spatial Denoised:", mse_original_spatial)
print("MSE - Original vs Temporal Denoised:", mse_original_temporal)
print("MSE - Original vs DCT Denoised:", mse_original_dct)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
PSNR - Original vs Spatial Denoised: 22.618301
PSNR - Original vs Temporal Denoised: 23.804422
PSNR - Original vs DCT Denoised: 36.16873
SSIM - Original vs Spatial Denoised: 0.58703583
SSIM - Original vs Temporal Denoised: 0.5445469
SSIM - Original vs DCT Denoised: 0.9710502
MSE - Original vs Spatial Denoised: 357.0775
MSE - Original vs Temporal Denoised: 280.11725
MSE - Original vs DCT Denoised: 15.801986


In [ ]:


# PSNR values for each denoised video
psnr_values = {
    "Spatial Denoised": psnr_original_spatial,
    "Temporal Denoised": psnr_original_temporal,
    "DCT Denoised": psnr_original_dct
}

# Sort PSNR values in descending order
sorted_psnr = sorted(psnr_values.items(), key=lambda x: x[1], reverse=True)

# Generate explanation
explanation = "Comparing the PSNR values, the {} video has the highest PSNR value, indicating that it preserves image quality the best among the denoised videos. \nThe {} video comes next, followed by the {} video with the lowest PSNR value.\nTherefore, in terms of PSNR, the order of preference from best to worst would be: {}, {} > {}."

# Extract denoised methods
best_method = sorted_psnr[0][0]
better_method = sorted_psnr[1][0]
worst_method = sorted_psnr[2][0]

# Print explanation
print(explanation.format(best_method, sorted_psnr[1][0], sorted_psnr[2][0], best_method, better_method, worst_method))


Comparing the PSNR values, the DCT Denoised video has the highest PSNR value, indicating that it preserves image quality the best among the denoised videos. 
The Temporal Denoised video comes next, followed by the Spatial Denoised video with the lowest PSNR value.
Therefore, in terms of PSNR, the order of preference from best to worst would be: DCT Denoised, Temporal Denoised > Spatial Denoised.


In [ ]:
# SSIM values taken from the provided code

# SSIM values for each denoised video
ssim_values = {
    "Spatial Denoised": ssim_original_spatial,
    "Temporal Denoised": ssim_original_temporal,
    "DCT Denoised": ssim_original_dct
}

# Sort SSIM values in descending order
sorted_ssim = sorted(ssim_values.items(), key=lambda x: x[1], reverse=True)

# Generate explanation
explanation = "Comparing the SSIM values, the {} video has the highest SSIM value, indicating that it preserves image quality the best among the denoised videos. \nThe {} video comes next, followed by the {} video with the lowest SSIM value.\nTherefore, in terms of SSIM, the order of preference from best to worst would be: {}, {} > {}."

# Extract denoised methods
best_method = sorted_ssim[0][0]
better_method = sorted_ssim[1][0]
worst_method = sorted_ssim[2][0]

# Print explanation
print(explanation.format(best_method, sorted_ssim[1][0], sorted_ssim[2][0], best_method, better_method, worst_method))


Comparing the SSIM values, the DCT Denoised video has the highest SSIM value, indicating that it preserves image quality the best among the denoised videos. 
The Spatial Denoised video comes next, followed by the Temporal Denoised video with the lowest SSIM value.
Therefore, in terms of SSIM, the order of preference from best to worst would be: DCT Denoised, Spatial Denoised > Temporal Denoised.


In [ ]:


# MSE values for each denoised video
mse_values = {
    "Spatial Denoised": mse_original_spatial,
    "Temporal Denoised": mse_original_temporal,
    "DCT Denoised": mse_original_dct
}

# Sort MSE values in ascending order
sorted_mse = sorted(mse_values.items(), key=lambda x: x[1])

# Generate explanation
explanation = "Comparing the MSE values, the {} video has the lowest MSE value, indicating that it preserves image quality the best among the denoised videos. \nThe {} video comes next, followed by the {} video with the highest MSE value.\nTherefore, in terms of MSE, the order of preference from best to worst would be: {}, {} > {}."

# Extract denoised methods
best_method = sorted_mse[0][0]
better_method = sorted_mse[1][0]
worst_method = sorted_mse[2][0]

# Print explanation
print(explanation.format(best_method, sorted_mse[1][0], sorted_mse[2][0], best_method, better_method, worst_method))


Comparing the MSE values, the DCT Denoised video has the lowest MSE value, indicating that it preserves image quality the best among the denoised videos. 
The Temporal Denoised video comes next, followed by the Spatial Denoised video with the highest MSE value.
Therefore, in terms of MSE, the order of preference from best to worst would be: DCT Denoised, Temporal Denoised > Spatial Denoised.
